In [1]:
#Importing Basic Libraries 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
#program to display warning a message 
import warnings
# displaying the warning message 
warnings.filterwarnings('ignore')

#Import Scraping Libraries
import requests
import pandas as pd
import re
from bs4 import BeautifulSoup


header_ = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_5) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/12.1.1 Safari/605.1.15',
       'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
       'Referer': 'http://www.google.com/',
       'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
       'Accept-Encoding': 'none',
       'Accept-Language': 'en-US,en;q=0.8',
       'Connection': 'keep-alive'}

In [2]:
df= pd.read_csv(r"C:\Users\gouth\OneDrive\Desktop\ISB\FP1\UpdatedResumeDataSet.csv")
df.head()

,Category,Resume
0,Data Science,Skills * Programming Languages: Python (pandas...
1,Data Science,Education Details \r\nMay 2013 to May 2017 B.E...
2,Data Science,"Areas of Interest Deep Learning, Control Syste..."
3,Data Science,Skills â¢ R â¢ Python â¢ SAP HANA â¢ Table...
4,Data Science,"Education Details \r\n MCA YMCAUST, Faridab..."


In [3]:
categories_list = df["Category"].unique()

In [4]:
categories_list

array(['Data Science', 'HR', 'Advocate', 'Arts', 'Web Designing',
       'Mechanical Engineer', 'Sales', 'Health and fitness',
       'Civil Engineer', 'Java Developer', 'Business Analyst',
       'SAP Developer', 'Automation Testing', 'Electrical Engineering',
       'Operations Manager', 'Python Developer', 'DevOps Engineer',
       'Network Security Engineer', 'PMO', 'Database', 'Hadoop',
       'ETL Developer', 'DotNet Developer', 'Blockchain', 'Testing'],
      dtype=object)

In [5]:
jds_per_page = 20
number_of_pages = 2

jd_data = []
for category in categories_list:
    print(category)
    page_num = 0
    for i in range(0,number_of_pages):
        page_num = i+1
        print(page_num)
        category_term_list = category.split()

        category_term_count = len(category_term_list)

        search_term = ""
        for i in range(0,category_term_count):
            if category_term_count>1:
                if search_term=="":
                    search_term = category_term_list[i]
                else:
                    search_term = search_term + "%20" + category_term_list[i]
            else:
                search_term = category_term_list[0]


        search_string = "https://www.timesjobs.com/candidate/job-search.html?from=submit&actualTxtKeywords=" \
                        + search_term \
                        + "&searchBy=0&rdoOperator=OR&searchType=personalizedSearch&txtLocation=hyderabad&luceneResultSize=" \
                        + str(jds_per_page) \
                        + "&postWeek=60&txtKeywords=" \
                        + search_term \
                        + "&pDate=I&sequence=" \
                        + str(page_num) \
                        + "&startPage=1"

        page = requests.get(search_string,headers=header_)
        soup = BeautifulSoup(page.content, 'html.parser')

        results = soup.find("div",class_="inside-rhs main-search-block")
        job_box = results.find_all("li",class_="clearfix job-bx wht-shd-bx")


        for jd in job_box:
            item_link_comp = jd.find("a")
            jd_data.append((category,item_link_comp["href"]))

    

Data Science
1
2
HR
1
2
Advocate
1
2
Arts
1
2
Web Designing
1
2
Mechanical Engineer
1
2
Sales
1
2
Health and fitness
1
2
Civil Engineer
1
2
Java Developer
1
2
Business Analyst
1
2
SAP Developer
1
2
Automation Testing
1
2
Electrical Engineering
1
2
Operations Manager
1
2
Python Developer
1
2
DevOps Engineer
1
2
Network Security Engineer
1
2
PMO
1
2
Database
1
2
Hadoop
1
2
ETL Developer
1
2
DotNet Developer
1
2
Blockchain
1
2
Testing
1
2


In [6]:
jd_links_df = pd.DataFrame(jd_data, columns = ['Category','Link'])

In [7]:
jd_links_df.head()

,Category,Link
0,Data Science,https://www.timesjobs.com/job-detail/data-scie...
1,Data Science,https://www.timesjobs.com/job-detail/manager-d...
2,Data Science,https://www.timesjobs.com/job-detail/data-scie...
3,Data Science,https://www.timesjobs.com/job-detail/data-scie...
4,Data Science,https://www.timesjobs.com/job-detail/data-scie...


In [8]:
jd_links_df.to_csv(r"C:\Users\gouth\OneDrive\Desktop\ISB\FP1\jd_links.csv")

In [9]:
full_job_details_array = []
for ind in jd_links_df.index:

    Category = jd_links_df['Category'][ind]
    JD_Link = jd_links_df['Link'][ind]
    jd_page = requests.get(JD_Link,headers=header_)
    jd_soup = BeautifulSoup(jd_page.content, 'html.parser') 
    
    
    jd_header = jd_soup.find("div",class_="jd-header wht-shd-bx")
    job_title_element = jd_header.find("h1",class_="jd-job-title")
    job_title_text = job_title_element.get_text()
    job_title = re.sub(r'[\t\r\n]', '', job_title_text).strip()
    
    print(job_title)


    job_title_sub_element = jd_header.find("h2")
    job_title_sub = job_title_sub_element.get_text().strip()


    job_header_details_element = jd_header.find("ul",class_="top-jd-dtl clearfix")

    job_header_details_element_array = job_header_details_element.find_all("li")

    job_header_details_elements = []
    for elem in job_header_details_element_array:  
        elem_cleaned = re.sub(r'\s\s+', ' ', elem.get_text()).strip()
        job_header_details_elements.append(elem_cleaned)
        
    experience_level = job_header_details_elements[0]
    salary = job_header_details_elements[1]
    location = job_header_details_elements[2]

    main_job_desc_block = jd_soup.find("div",class_="jd-desc job-description-main")
    job_desc_raw = main_job_desc_block.get_text()

    sub_job_desc_block = jd_soup.find("div",class_="job-basic-info")
    sub_job_desc_elements = sub_job_desc_block.find_all("li", class_="clearfix")

    job_cat_details_dict = {}

    for i in sub_job_desc_elements:

        if i.find("label") is not None:

            cat_label = i.find("label").get_text()
            i.label.unwrap()

        cat_detail = re.sub(r'\s\s+', ' ', i.get_text()).strip()
        cat_detail = re.sub(cat_label,'',cat_detail).strip()

        job_cat_details_dict[cat_label] = cat_detail
    
    
    full_job_details_array.append((Category, JD_Link, job_title, job_title_sub, experience_level, salary, location, job_desc_raw, job_cat_details_dict))



directions_walk    Data Science Internship in Mumbai
Manager, Data Science
Data Science Specialist
Data Science Analyst
Data Science Engineer
directions_walk    Data Science Internship in Mumbai
Data Science Analyst
Lead - Data Science
Lead - Data Science
Senior Manager - Data Science
Senior Manager - Data Science
Manager - Data Science
Analyst - Data Science
Senior Engineer - Data Science
Analyst - Data Science
Lead (Data Science Engineer)
Senior Analyst, Data Science
Engineer - Data Science
Engineer - Data Science
Manager - Data Science
Data Science - The Edge
Marketing Analytics with Data Science Lead
Executive Manager - Data Science - wiq
Data Scientists
Internship in Clinical Research / Clinical Data Management / pv
Data Research Executive:
Data Analyst
Data Entry Executives (Graduates only) US Health Process
Data Entry Operator,"Data Entry Executive"
This job is very special no boss no time limitations
KOLKATA MUMBAI CHENNAI BHUBANESWAR BANGALORE etc
Chennai, Bangalore/Bengaluru 

Full Stack Java Developer
Sr. Java Developer
Senior Java Developer
Position: Senior Java developer
Senior Java Developer
Senior Java Developer
Senior Java Developer
Sr. Java Developer
Specialist - Core Java Developer
Senior Java Developer
Jr. Java Developer
Sr. Java Developer
Hadoop  Java Developer/Lead
Junior Java Developer.
Lead Java Developer
Senior Java Developer
Mid Level Java Developer
Java Developer/ Senior Developer
Senior Java Developer
Business Analyst
Business Analyst
Business Analyst
Business Analyst
Business Analyst
Business Analyst
Business Analyst
Business Analyst
Business Analyst
Business Analyst
Business Analyst
Business Analyst
Business Analyst
Business Analyst
Business Partner,Business Analyst,Senior Business Analyst
Business Analyst (SAP FICO)
MIS / Business Analyst
Business Analyst and Project Manager
Business Analyst and Project Manager
Business Analyst and Project Manager
Business Analyst- IT
Business Analyst-Duckcreek-Billing
Business analyst (IT)
Senior Consult

Hiring for Architecture Related Profiles
Engineering Jobs
Hiring for Engineering Profiles
Civil Engineer
Newly clevery work for smarties
KOLKATA MUMBAI CHENNAI BHUBANESWAR BANGALORE etc
Team Lead Java - J46975
Network Security Engineer
Sr. Information Security Engineer
Application Security Engineer
Network Engineer,System Administrator,System Engineer
IT Hardware - Profiles Related Jobs
internship
Team Lead Java - J46975
Electrical Engineer
Product Safety and Security Engineer
Design Engineer,Senior Design Engineer
Security Engineer - IAM ( OKTA )
Application Security Engineer
Service Delivery- Manager
Network Engineer
IT Manager
Business Process Analyst,Network Executive,Network Engineer
Network Engineer L2
Network Engineer
Network Engineer
Network Marketers to earn Extra Money from Home Comfort.
Network Engineer
Network Administrator
Network Engineer
Network Engineer
Network Architect
Network Engineer
Health Unit Coordinator
Azure Security SME
directions_walk    project engineer
Use 

In [10]:
jd_full_columns = ["Category","JD_Link", "Job_title", "Job_title_sub", "Experience_level", "Salary", "Location", 
                   "Job_desc_raw", "Job_cat_details_dict"]

In [11]:
jd_full_df = pd.DataFrame(full_job_details_array, columns = jd_full_columns)

jd_full_df.head()

,Category,JD_Link,Job_title,Job_title_sub,Experience_level,Salary,Location,Job_desc_raw,Job_cat_details_dict
0,Data Science,https://www.timesjobs.com/job-detail/data-scie...,directions_walk Data Science Internship in ...,Maxgen Technologies\r\n \t\t\t\r\...,card_travel 0 to 1 yrs,₹ As per Industry Standards,"location_on Ahmedabad (Gujarat), Bengaluru/ Ba...",\nJob Description\nMaxgen technologies is an i...,{'Job Function:': 'IT Software : Software Prod...
1,Data Science,https://www.timesjobs.com/job-detail/manager-d...,"Manager, Data Science",Electrobrain modern technologies pvt ltd,card_travel 2 to 7 yrs,₹ Rs 4.20 - 6.40 Lacs p.a.,"location_on Gurgaon (Haryana), Hyderabad/ Secu...",\nJob Description\n\nPerform deep dive analyse...,{'Job Function:': 'Accounting/Tax/Company Secr...
2,Data Science,https://www.timesjobs.com/job-detail/data-scie...,Data Science Specialist,Electrobrain modern technologies pvt ltd,card_travel 4 to 9 yrs,₹ Rs 4.25 - 6.70 Lacs p.a.,"location_on Gurgaon (Haryana), Hyderabad/ Secu...","\nJob Description\nManage, architect, and anal...",{'Job Function:': 'IT Software : Software Prod...
3,Data Science,https://www.timesjobs.com/job-detail/data-scie...,Data Science Analyst,Electrobrain modern technologies pvt ltd,card_travel 2 to 7 yrs,₹ Rs 3.20 - 5.50 Lacs p.a.,"location_on Gurgaon (Haryana), Hyderabad/ Secu...",\nJob Description\n\n3+ years of relevant data...,{'Job Function:': 'IT Software : Software Prod...
4,Data Science,https://www.timesjobs.com/job-detail/data-scie...,Data Science Engineer,Electrobrain modern technologies pvt ltd,card_travel 2 to 6 yrs,₹ Rs 3.20 - 6.60 Lacs p.a.,"location_on Gurgaon (Haryana), Hyderabad/ Secu...",\nJob Description\n. Requirements: Should have...,{'Job Function:': 'IT Software : Software Prod...


In [12]:
jd_full_export_df = pd.concat([jd_full_df.drop(['Job_cat_details_dict'], axis=1), jd_full_df['Job_cat_details_dict'].apply(pd.Series)], axis=1)

In [13]:
jd_full_export_df.to_csv(r"C:\Users\gouth\OneDrive\Desktop\ISB\FP1\jd_full_raw_data.csv")